# 概率面试题精讲

![title](./images/probability_001.png)

![title](./images/probability_002.png)

### 题外话：
- 随机数：
    - 随机数生成并不容易
        - “随机性”和“不可预测性”
            - 固定m，自然数n % m 是“均匀”的，具有一定随机性，但密码学不采用它。
    - 一般假设已有一个均匀的随机数生成器
- 期望的计算： 
    - 一般转化为方程组：  
        $ E(A) = E(A_1) * p_1 +  E(A_2) * p_2 + ...+ 1$

![title](./images/probability_003.png)

 # 例1：关于“独立”的理解 
 
$X_1, X_2$都是二元随机变量，取值0和1的概率各一半，则$X_3 = X_1\ xor\  X_2$，它与$X_1, X_2$独立。

xor 异或：

<img src="./images/probability_004.png" width="40%">

分析：  
看起来，$X_3$与$X_1, X_2$都有关系，但实际上$X_3$与$X_1, X_2$都是独立的。  

很简单，我们可以用枚举法，也就是对应上面的“真值表”。

枚举法：{000,011,101,110}。

可见当$X_1=0,1$时各有一半情况$X_3 = 0,1$。  
即：  
当$X_1=0$时，$X_3 =0,1$；  
当$X_1=1$时，$X_3 =0,1$；  

即$X_1$和$X_3$是独立的。     
同理，$X_2$和$X_3$是独立的。    

这个结论与$X_3 = X_1\ xor\  X_2$的式子，看起来有点“反直觉”。  
但实际上是相互独立的。  




关于独立：  
### 不能凭感觉，需要根据定义来判断。
如果A和B是独立的，$P(A\bigcap B) = P(A)*P(B)$

p(AB)求的是A，B同时发生的概率，但不要求A,B独立，A,B如果独立，那么p(AB)=p(A)p(B)，不独立就不能写成这样了。  

而条件概率就是研究A,B一般不独立的情况的，条件概率的公式P(B|A)=p(AB)/p(A)，假如A,B独立，那么P(B|A)也就是p(B)了，代回条件概率的公式，可以看出它就是独立事件的公式p(AB)=p(A)p(B)了。


---

# 例2：构造随机数发生器
假设一个随机数发生器rand7均匀产生1到7之间的随机整数，如何构造rand10，均匀产生1-10之间的随机整数?

分析：   
关键在于，不想要的数可以扔，要保证**“等概率”**。

方法1：（笨方法）
1-7之间有4个奇数，3个偶数，我们扔掉一个奇数，比如7，这样就剩余3个奇数，3个偶数。
这样奇数（看作1）和偶数（看作0）产生的概率相同--我们构造了一个0-1整数的均匀产生器。   
有了0，1就有了一切，可以用二进制表示想要表示的值。   

用它产生4个bit，即$2^4 = 16$，对应表示0-15之间的整数，共16个数字。    
并且每个数字产生的概率也是**等概率**的--因为每个bit是0或1的概率都是$\frac{1}{2}$。  
那么，后面把不想要的数字扔掉即可，扔掉11-15的整数和0，保留1-10就可以了。

In [61]:
# 假定rand7()为题目中提供的随机整数1-7的生成器
def rand7():
    import random
    return random.randint(1,7)
    
# 方法1：笨方法0-1二进制生成器
def binary_rand():
    if rand7() == 7:
        return binary_rand()
    elif rand7() % 2 ==0:
        return 0
    else:
        return 1


# 生成四位二进制数并转换为10进制，去除不要的整数0和11-15。
def rand10():
    binary_number = []
    for i in range(4):
        binary_number.append(str(binary_rand()))
    
    binary_number_str = ''.join(binary_number) 
    decimal_number = int(binary_number_str, 2)
    
    if decimal_number in [0,11,12,13,14,15]:
        return rand10()
    else:
        return decimal_number

# rand10随机生成器验证
for i in range(10):
    print(rand10())

1
9
2
6
9
2
6
6
4
4


方法2：（聪明一点）   
使用“七进制”：   
我们把1-7减去7，变成0-6。   
产生一个两位的“七进制”数，对应0-48，都是均匀产生的，每个数产生的概率都是$\frac{1}{49}$。   
和上面方法一的策略相同，把不想要的数字扔掉。   
我们把40-48扔掉（因为这里只有9个数，前面的数字都是0-9,10-19,20-29,30-39,是十个数一组，个位数上都是0-9），其余按照个位数字分类，0-9对应我们要的1-10。

In [93]:
def sevenary_rand7():
    import random
    two_sevenary_number  = (random.randint(1,7) - 1)*7+random.randint(1,7) -1
    if two_sevenary_number in range(40,49):
        return sevenary_rand7()
    else:
        return two_sevenary_number
    
def sevenary_rand10():
    return sevenary_rand7() % 10 + 1
    
sevenary_rand10()

3

### 关键问题：
- 保证均匀，才能扔掉。  
    假定有一个rand2()，均匀产生1或2。  
    rand2() + rand2() - 1 并不是均匀的1-3，其中1和3产生的概率是$\frac{1}{4}$，2产生的概率是$\frac{1}{2}$。
    
    下面是枚举法的结果：   
    {1+1-1, 1+2-1, 2+1-1, 2+2-1}= {1, 2, 2, 3}
    


所以rand2()+rand2()-1的结果不能随便扔，因为它的结果不能保证等概率。

### 分析：
一个实验成功的概率是p，则不断实验直到一次成功的期望次数是$\frac{1}{p}$。
假定x是期望，那么$p*1$表示第一次就成功的概率
$$ p*1 + (1-p)*(x+1) = x$$

验证：将$x=\frac{1}{p}$代入，发现等式成立。
$$ p*1 + (1-p)*(x+1) = x$$
$$ p+x-px+1-p=x$$
$$ p+\frac{1}{p}-p*\frac{1}{p}+1-p = \frac{1}{p}$$
$$ p+\frac{1}{p}-1+1-p = \frac{1}{p}$$

## 补：期望次数
第n次才成功的概率为：   
$P_n = p*(1-p)^{n-1}$

所以期望次数为：   
$E = \sum (P_n *n)$
$=\sum (n*p*(1-p)^{n-1})$
$=p*\sum(n*(1-p)^{n-1})$

因为$\sum (n*(1-p)^{n-1}) = (\sum (1-p)^n)' =(\frac{1-p}{p})' = \frac{1}{p^2}$
代入上式得：
$E= p*\frac{1}{p^2} =\frac{1}{p}$


举个例子：  
扔硬币，假定扔到正面的概率为$\frac{1}{2}$，那么不断试验一次成功扔到正面的期望次数为$\frac{1}{\frac{1}{2}} = 2$次。  

请计算方法1和方法2的期望循环次数：  
方法1的期望循环次数：
$\frac{112}{15}$

方法2的期望循环次数：
$\frac{49}{20}$

----

# 例3：不均匀随机数发生器构造均匀
一个随机数发生器，不均匀，以概率p产生0，以(1-p)产生1, (0<p<1)，构造一个均匀的随机数发生器（算法导论）   

分析：   
产生两次，(0,1)的概率与(1,0)的概率相同，都是$p*(1-p)$。